<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 11, 2025</h2>

<p style="font-size:16pt">
Calculation of the optimized entropy

In [23]:
import numpy as np
import matplotlib.tri as mtri
from scipy.spatial import Delaunay
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import networkx as nx
import scipy
import seaborn as sns
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.decomposition import PCA
from functools import reduce
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
from astropy.io import ascii
from itertools import combinations
from concurrent.futures import ProcessPoolExecutor
from functools import partial

In [24]:
%%time
rosettes = list(range(20))
data = {}

for number_rosette in range(20):
    file = f"data_rosette/LRG_{number_rosette}_clustering_data.ecsv"
    table = Table.read(file, format="ascii.ecsv") 
    subset = table[['TARGETID','RA', 'DEC', 'Z','x','y','z']].to_pandas()
    data[f"data_{number_rosette}"] = subset
    data[f"data_{number_rosette}"]['type'] = 'data'

CPU times: total: 8.19 s
Wall time: 13.7 s


In [ ]:
def parameter_r(data_all):
    df_tri = data_all[['x', 'y', 'z']].values
    tri = Delaunay(df_tri)

    G = nx.Graph()
    ids = data_all['TARGETID'].values
    types = data_all['type'].values

    # Crear nodos con atributos
    for coords, tipo, id_ in zip(df_tri, types, ids):
        G.add_node(id_, pos=tuple(coords), type=tipo)

    # Conectar nodos usando TARGETID
    G.add_edges_from(
        (ids[simplex[i]], ids[simplex[j]])
        for simplex in tri.simplices
        for i in range(3)
        for j in range(i + 1, 4)
    )

    # Calcular degree
    degree_dict = dict(G.degree())
    data_all['degree'] = data_all['TARGETID'].map(degree_dict)

    # Contar vecinos por tipo
    n_data_dict = {}
    n_random_dict = {}

    for node in G.nodes:
        neighbors = list(G.neighbors(node))
        neighbor_types = [G.nodes[n]['type'] for n in neighbors]
        n_data_dict[node] = neighbor_types.count('data')
        n_random_dict[node] = neighbor_types.count('rand')

    data_all['N_data'] = data_all['TARGETID'].map(n_data_dict)
    data_all['N_random'] = data_all['TARGETID'].map(n_random_dict)

    # Calcular r
    total = data_all['N_data'] + data_all['N_random']
    with np.errstate(divide='ignore', invalid='ignore'):
        r = np.where(total > 0, (data_all['N_data'] - data_all['N_random']) / total, 0)

    data_all['r'] = r

    return data_all


In [54]:
def compute_r(df):
    coords = df[['x', 'y', 'z']].values
    types = df['type'].values

    tri = Delaunay(coords)

    # Adjacency list for neighbors
    neighbors = {i: set() for i in range(len(coords))}
    for simplex in tri.simplices:
        for i, j in combinations(simplex, 2):
            neighbors[i].add(j)
            neighbors[j].add(i)

    r = np.zeros(len(coords), dtype=float)

    for i, nbrs in neighbors.items():
        nbrs = list(nbrs)
        type_neighbors = types[nbrs]  # extraer los tipos de los vecinos
        n_data = np.sum(type_neighbors == 'data')
        n_rand = len(nbrs) - n_data

        if (n_data + n_rand) > 0:
            r[i] = (n_data - n_rand) / (n_data + n_rand)
        else:
            raise ValueError(f'No neighbors for point {i} in the triangulation.')

    out = df.copy()
    out['r'] = r
    return out


In [48]:
def classification(data,number_rand):

    data.loc[(data['r'] >= -1.0) & (data['r'] <= -0.9), f'class_{number_rand}'] = 'void'
    data.loc[(data['r'] >  -0.9) & (data['r'] <=  0.0), f'class_{number_rand}'] = 'sheet'
    data.loc[(data['r'] >   0.0) & (data['r'] <=  0.9), f'class_{number_rand}'] = 'filament'
    data.loc[(data['r'] >   0.9) & (data['r'] <=  1.0), f'class_{number_rand}'] = 'knot'

    data.sort_values('z', inplace=True)

    return data

In [49]:
def entropy(data):
    count = data[data.columns].apply(lambda row: row.value_counts(), axis=1)
    probabilidades = count.div(number_random)
    sum_entropy = probabilidades.apply(lambda row: np.sum(row[row > 0] * np.log2(row[row > 0])), axis=1)
    entropy = -(1/np.log2(4))*sum_entropy
    entropy = np.abs(entropy)
    return entropy,probabilidades

In [64]:
%%time
rand = {}
rand_list = {}
df_merge = {}
df_entropy = {}
number_random = 100

for i in range(20):
    rand_list[i] = []
    print(f'Rosette {i}')
    
    for j in range(number_random):

        #Read files
        file = f'rand_rosette/LRG_rosette_{i}_random_{j}.ecsv'
        table = Table.read(file, format="ascii.ecsv") 
        subset = table[['TARGETID', 'RA', 'DEC', 'Z', 'x', 'y', 'z']].to_pandas()
        subset['type'] = 'rand'

        #Concat real and random data
        df_concat = pd.concat([subset, data[f'data_{i}'].copy().assign(type='data')], ignore_index=True)

        #Parameter r and classification
        #data_with_r = parameter_r(df_concat)
        data_with_r = compute_r(df_concat)
        data_with_class = classification(data_with_r, j)
        
        rand_list[i].append(data_with_class)

    #Merge of the same rossete
    df_merge[i] = data[f'data_{i}'][['TARGETID']].copy()
    for j in range(number_random):
        df_j = rand_list[i][j][['TARGETID', f'class_{j}']]
        df_merge[i] = df_merge[i].merge(df_j, on='TARGETID', how='left')

    #Entropy
    reset = entropy(df_merge[i].set_index('TARGETID'))
    reset_entropy = reset[0].rename('entropy').reset_index()
    df_merge[i] = df_merge[i].merge(reset_entropy, on='TARGETID', how='left')
    df_entropy[i] = df_merge[i][['TARGETID','entropy']].copy()

    #Save file of entropy
    table_entropy = Table.from_pandas(df_entropy[i])
    filename = f"entropy_rosette/LRG_entropy_rosette_{i}.ecsv"
    ascii.write(table_entropy, filename, format='ecsv', overwrite=True)

Rosette 0
Rosette 1
Rosette 2
Rosette 3
Rosette 4
Rosette 5
Rosette 6
Rosette 7
Rosette 8
Rosette 9
Rosette 10
Rosette 11
Rosette 12
Rosette 13
Rosette 14
Rosette 15
Rosette 16
Rosette 17
Rosette 18
Rosette 19
CPU times: total: 41min 3s
Wall time: 43min 17s
